<a href="https://colab.research.google.com/github/manuel-suarez/tensorflow_examples/blob/main/TensorFlow_examples_Writing_a_training_loop_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Writing a training loop from scratch](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch?hl=en)